# 说明

1. 该代码使用`tensorflow-gpu` `v1.2.1`
2. 本代码中有大量推荐参数，请先按照推荐参数排列组合进行训练

In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import os

# Parameter Configuration

In [ ]:
# network parameters
n_step = 100  # input data length, for 1 second data with sampling frequency 100Hz, n_step=100
n_width = 3  # each sensor has 3 axes
n_channel = 2  # 2 sensors
n_hidden = 128  # hidden layer neuron. Recommended values 32, 64, 128

# learning parameters
learning_rate = 0.0025  # recommended values 0.0025, 0.005, 0.01
l2_lambda = 0.002  # recommended values 0.00025, 0.0005, 0.001, 0.002
dropout_prob = 0.5
training_iters = 300  # 300 rounds
batch_size = 200
stop_loss = 0.15
# directory to store log, including loss and grad_norm of generator and critic
dir_mark = 'lstm_demo_model'
log_dir = './log_sensor/' + dir_mark
ckpt_dir = './ckpt_sensor/' + dir_mark
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

# Network Sturcture

In [ ]:
# tf graph input
x = tf.placeholder("float", [None, n_step, n_width, n_channel])
y = tf.placeholder("float", [None, n_class])
keep_prob = tf.placeholder(tf.float32)

# define weights
weights = {
    # Hidden layer weights
    'hidden': tf.Variable(tf.random_normal([n_width * n_channel, n_hidden]), name='weight_hidden'),
    # 3*n_class inputs, n_class output(class prediction)
    'out': tf.Variable(tf.random_normal([n_hidden, n_class]), name='weight_out')
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden]), name='biases_hidden'),
    'out': tf.Variable(tf.random_normal([n_class]), name='biases_out')
}


def stackLSTM(x, weights, biases, keep_prob):
    with tf.variable_scope('trans'), tf.name_scope('trans'):
        x_trans = tf.reshape(x, [-1, n_step, n_width * n_channel])
        x_trans = tf.transpose(x_trans, [1, 0, 2])
        x_trans = tf.reshape(x_trans, [-1, n_width * n_channel])
        # Linear activation
        x_act = tf.nn.relu(
            tf.matmul(x_trans, weights['hidden']) + biases['hidden'])
        # Split data because rnn cell needs a list of inputs for the RNN inner loop
        x_split = tf.split(x_act, n_step, 0)
    with tf.variable_scope('stack_LSTM'), tf.name_scope('stack_LSTM'):
        # Define two stacked LSTM cells
        lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(
            n_hidden, forget_bias=1.0, state_is_tuple=True)
        lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(
            n_hidden, forget_bias=1.0, state_is_tuple=True)
        lstm_cells = tf.contrib.rnn.MultiRNNCell(
            [lstm_cell_1, lstm_cell_2], state_is_tuple=True)
        # Get LSTM cell output
        outputs, states = tf.contrib.rnn.static_rnn(
            lstm_cells, x_split, dtype=tf.float32)
    with tf.variable_scope('output'), tf.name_scope('output'):
        # Get last time step's output feature for a "many to one" style classifier
        lstm_last_output = outputs[-1]
        batchnorm = tf.contrib.layers.batch_norm(lstm_last_output)
        dropout = tf.layers.dropout(batchnorm, rate=keep_prob)
        out = tf.matmul(dropout, weights['out']) + biases['out']
    return out


pred = stackLSTM(x, weights, biases, keep_prob)

# Define loss and optimizer
regularization = l2_lambda * (tf.nn.l2_loss(
    weights['hidden']) + tf.nn.l2_loss(weights['out']))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=pred, labels=y)) + regularization
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

# Initializing the variables
init = tf.global_variables_initializer()

# 训练过程设计

```
training_iters = 5000000*2 # 300 rounds
batch_size = 200
display_step = 10
```
step的上限是20000， 然而屏幕输出的上限是10000000

- 每200步记录元信息，其它情况正常训练
- 每10个step输出一次训练集的准确率
- 当准确率高于0.95是，保存模型，停止训练
- 每300个step输出测试集准确率
- 每1000个step保存一下模型

In [ ]:
merged_all = tf.summary.merge_all()
saver = tf.train.Saver()
# Launch the graph
with tf.Session(config=config) as sess:
    sess.run(init)
    summary_train_writer = tf.summary.FileWriter(
        log_dir + '/train', sess.graph)
    summary_test_writer = tf.summary.FileWriter(log_dir + '/test')
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        temp = (step - 1) * batch_size
        bind = temp % trainSet_size
        eind = (temp + batch_size) % trainSet_size
        if (eind > bind):
            batch_x = xx[bind: eind, :, :]
            batch_y = yy[bind: eind, :]
        else:
            batch_x = np.vstack((xx[bind:, :, :], xx[:eind, :, :]))
            batch_y = np.vstack((yy[bind:, :], yy[:eind, :]))
        # Run optimization op (backprop)
        if step % 200 == 0:
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, _ = sess.run([merged_all, optimizer], feed_dict={x: batch_x, y: batch_y, keep_prob: dropout_prob},
                                  options=run_options, run_metadata=run_metadata)
            summary_train_writer.add_summary(summary, step)
            summary_train_writer.add_run_metadata(
                run_metadata, 'train_metadata {}'.format(step), step)

        else:
            summary, _ = sess.run([merged_all, optimizer], feed_dict={
                                  x: batch_x, y: batch_y, keep_prob: dropout_prob})
            summary_train_writer.add_summary(summary, step)
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 x: batch_x, y: batch_y, keep_prob: 1.})
            print("Iter " + str(step * batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) +
                  ", Training Accuracy= " + "{:.5f}".format(acc))
            if loss <= stop_loss:
                test_data = xx_test
                test_label = yy_test
                test_acc = sess.run(accuracy, feed_dict={
                                    x: test_data, y: test_label, keep_prob: 1.})
                print("Testing Accuracy:", test_acc)
                if test_acc >= 0.95:
                    save_path = saver.save(
                        sess, ckpt_dir + '/discriminator_birnn.model')
                    print("model is saved at: %s" % save_path)
                    summary_train_writer.close()
                    summary_test_writer.close()
                    sess.close()
        step += 1
        if (step % 100 == 0):
            summary, acc = sess.run([merged_all, accuracy], feed_dict={
                                    x: xx_test, y: yy_test, keep_prob: 1.})
            summary_test_writer.add_summary(summary, step)
            print("Testing Accuracy:", acc)
        if (step % 1000 == 0):
            save_path = saver.save(
                sess, ckpt_dir + '/saver_lstm_tensorflow_' + str(step) + '.model')
            print("model is saved at: %s" % save_path)

    summary_train_writer.close()
    summary_test_writer.close()
    print("Optimization Finished!")

    test_data = xx_test
    test_label = yy_test
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={
          x: test_data, y: test_label, keep_prob: 1.}))

    save_path = saver.save(
        sess, ckpt_dir + '/saver_tensorflow_' + str(step) + '.model')
    print("model is saved at: %s" % save_path)